Fashion-MNIST is a dataset of Zalando's article images—consisting of a training set of 60,000 examples and a test set of 10,000 examples. Each example is a 28x28 grayscale image, associated with a label from 10 classes.  

In this assignment, we
- build a MLP classifier for the Fashion-MNIST dataset.
- use PCA to reduce the dimensionality of the dataset, and make sure we preserve 95% of the explained variance. (20 points)
- train a classifier using the dimensionality reduced dataset with the same network toplogy as the previous classfier,  and compare the classification accuracy result with the one using the original dataset. (10 points)
- check whether we observe anything surprising. (10 points)
- follow and improve the example from the text to fine tune the neural network hyperparameters using RandomizedSearchCV. Note to use the dataset after the PCA step. This will make the search less time consuming. (40 points)
- report the test result using the best model obtained from the randomized search. Show the summary of the model. Compare this result with the previous results. (20 points)

In [163]:
import numpy as np
import os

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

Read the data.

In [164]:
import tensorflow as tf
fashion_mnist = tf.keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()

Set random seeds.

In [165]:
np.random.seed(42)
tf.random.set_seed(42)

In [166]:
from keras import models
from keras import layers

network = models.Sequential()
network.add(layers.Dense(64, activation='relu', input_shape=(28 * 28,)))
network.add(layers.Dense(10, activation='softmax'))

In [167]:
network.compile(optimizer='rmsprop',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

In [168]:
X_train_full = X_train_full.reshape((60000, 28 * 28))
X_train_full = X_train_full.astype('float32') / 255

X_test = X_test.reshape((10000, 28 * 28))
X_test = X_test.astype('float32') / 255

In [169]:
from tensorflow.keras.utils import to_categorical

y_train_full = to_categorical(y_train_full)
y_test = to_categorical(y_test)

In [170]:
X_valid = X_train_full[:5000]
y_valid = y_train_full[:5000]
X_train = X_train_full[5000:]
y_train = y_train_full[5000:]

In [171]:
network.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=20, batch_size=128)

Epoch 1/20
430/430 [==============================] - 2s 4ms/step - loss: 0.6110 - accuracy: 0.7890 - val_loss: 0.4479 - val_accuracy: 0.8508
Epoch 2/20
430/430 [==============================] - 1s 3ms/step - loss: 0.4438 - accuracy: 0.8441 - val_loss: 0.3987 - val_accuracy: 0.8648
Epoch 3/20
430/430 [==============================] - 1s 3ms/step - loss: 0.3999 - accuracy: 0.8566 - val_loss: 0.4019 - val_accuracy: 0.8560
Epoch 4/20
430/430 [==============================] - 1s 3ms/step - loss: 0.3713 - accuracy: 0.8674 - val_loss: 0.3754 - val_accuracy: 0.8616
Epoch 5/20
430/430 [==============================] - 1s 3ms/step - loss: 0.3528 - accuracy: 0.8731 - val_loss: 0.3531 - val_accuracy: 0.8754
Epoch 6/20
430/430 [==============================] - 1s 3ms/step - loss: 0.3355 - accuracy: 0.8788 - val_loss: 0.3412 - val_accuracy: 0.8810
Epoch 7/20
430/430 [==============================] - 1s 3ms/step - loss: 0.3222 - accuracy: 0.8836 - val_loss: 0.3387 - val_accuracy: 0.8820
Epoch 

In [172]:
test_loss, test_acc = network.evaluate(X_test, y_test)

313/313 [==============================] - 1s 2ms/step - loss: 0.3741 - accuracy: 0.8694


In [173]:
print('test_acc:', test_acc)

test_acc: 0.8694000244140625


In [174]:
test_acc_withoutPCA = test_acc
test_acc_withoutPCA

0.8694000244140625

Now we read the data again.

In [175]:
import tensorflow as tf
fashion_mnist = tf.keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()

Use the same random seeds.

In [176]:
np.random.seed(42)
tf.random.set_seed(42)

In [177]:
X_train_full.shape

(60000, 28, 28)

In [178]:
X_test.shape

(10000, 28, 28)

In [179]:
X_train_full = X_train_full.reshape((60000,28*28))
X_test = X_test.reshape((10000,28*28))

Conduct fit and transform on X_train_full using PCA. (10 points)

In [180]:
from sklearn.decomposition import PCA

# fill in code here
pca = PCA(n_components=0.95)
X_train_reduced_full = pca.fit_transform(X_train_full)

Transform X_test using the PCA. (10 points)

In [181]:
# fill in code here
X_test_reduced = pca.transform(X_test)

Fill in the input_shape in the following code. (10 points)

In [182]:
from keras import models
from keras import layers

network = models.Sequential()
# fill in code
input_shape = (X_train_reduced_full.shape[1],)
network.add(layers.Dense(64, activation='relu', input_shape=input_shape))
network.add(layers.Dense(10, activation='softmax'))

In [183]:
network.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_22 (Dense)            (None, 64)                12032     
                                                                 
 dense_23 (Dense)            (None, 10)                650       
                                                                 
Total params: 12682 (49.54 KB)
Trainable params: 12682 (49.54 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [184]:
network.compile(optimizer='rmsprop',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

In [185]:
X_train_reduced_full = X_train_reduced_full.astype('float32') / 255
X_test_reduced = X_test_reduced.astype('float32') / 255

In [186]:
from tensorflow.keras.utils import to_categorical

y_train_full = to_categorical(y_train_full)
y_test = to_categorical(y_test)

In [187]:
X_valid_reduced = X_train_reduced_full[:5000]
y_valid = y_train_full[:5000]
X_train_reduced = X_train_reduced_full[5000:]
y_train = y_train_full[5000:]

In [188]:
X_train_reduced_full.shape

(60000, 187)

In [189]:
y_train.shape

(55000, 10)

In [190]:
network.fit(X_train_reduced, y_train, validation_data=(X_valid_reduced, y_valid), epochs=20, batch_size=128)

Epoch 1/20
430/430 [==============================] - 2s 4ms/step - loss: 0.6870 - accuracy: 0.7865 - val_loss: 0.4109 - val_accuracy: 0.8602
Epoch 2/20
430/430 [==============================] - 1s 3ms/step - loss: 0.3875 - accuracy: 0.8623 - val_loss: 0.3678 - val_accuracy: 0.8708
Epoch 3/20
430/430 [==============================] - 1s 3ms/step - loss: 0.3504 - accuracy: 0.8740 - val_loss: 0.3529 - val_accuracy: 0.8752
Epoch 4/20
430/430 [==============================] - 1s 3ms/step - loss: 0.3270 - accuracy: 0.8821 - val_loss: 0.3365 - val_accuracy: 0.8770
Epoch 5/20
430/430 [==============================] - 2s 4ms/step - loss: 0.3098 - accuracy: 0.8887 - val_loss: 0.3282 - val_accuracy: 0.8816
Epoch 6/20
430/430 [==============================] - 2s 4ms/step - loss: 0.2952 - accuracy: 0.8938 - val_loss: 0.3221 - val_accuracy: 0.8822
Epoch 7/20
430/430 [==============================] - 1s 3ms/step - loss: 0.2828 - accuracy: 0.8985 - val_loss: 0.3143 - val_accuracy: 0.8878
Epoch 

In [191]:
test_loss, test_acc = network.evaluate(X_test_reduced, y_test)

313/313 [==============================] - 1s 3ms/step - loss: 0.3434 - accuracy: 0.8835


In [192]:
print('test_acc:', test_acc)

test_acc: 0.8834999799728394


In [193]:
test_acc_PCA = test_acc
test_acc_PCA

0.8834999799728394

Compare these two accuracy results and check whether we see anything surprising. (10 points)

The accuracy result ...

In [195]:
print("Test Acc without PCA: ", test_acc_withoutPCA)
print("Test Acc PCA: ", test_acc_PCA)
print("\nDifference: ", test_acc_PCA - test_acc_withoutPCA)

Test Acc without PCA:  0.8694000244140625
Test Acc PCA:  0.8834999799728394

Difference:  0.014099955558776855


**Answer:** We can observe that we have a slight increase in accuracy for PCA over previous method. Using PCA method, we got around 1.4% higher accuracy.

In [196]:
np.random.seed(42)
tf.random.set_seed(42)

Modify the code provided by this module and use RandomizedSearchCV to find a model that beats the previous accuracy results. (40 points)

Hint: you can speed up the search by using n_jobs = 1 in RandomizedSearchCV.

In [197]:
X_valid = X_train_reduced[:5000]
y_valid = y_train[:5000]
X_train = X_train_reduced[5000:]
y_train = y_train[5000:]

In [198]:
X_train.shape, X_valid.shape

((50000, 187), (5000, 187))

In [199]:
y_train.shape, y_valid.shape, y_test.shape

((50000, 10), (5000, 10), (10000, 10))

In [200]:
from tensorflow import keras

keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [201]:
def build_model(n_hidden=1, n_neurons=128, learning_rate=3e-3, input_shape=(X_train_reduced.shape[1],)):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape=input_shape))
    for layer in range(n_hidden):
        # fill in code
        model.add(keras.layers.Dense(n_neurons, activation="relu"))
    # fill in code
    model.add(layers.Dense(10, activation='softmax'))
    optimizer = keras.optimizers.SGD(learning_rate=learning_rate)
    model.compile(optimizer='rmsprop',
                  # fill in code
                loss='categorical_crossentropy',
                metrics=['accuracy'])

    return model

In [202]:
from tensorflow import keras
from sklearn.base import BaseEstimator, RegressorMixin

# Modify the KerasRegressorWrapper class to accept hyperparameters
class KerasRegressorWrapper(BaseEstimator, RegressorMixin):
    def __init__(self, n_hidden=1, n_neurons=200, learning_rate=1e-3):
        self.n_hidden = n_hidden
        self.n_neurons = n_neurons
        self.learning_rate = learning_rate

    def fit(self, X, y, **kwargs):
        self.model = build_model(self.n_hidden, self.n_neurons, self.learning_rate)
        self.model.fit(X, y, **kwargs)
        return self

    def predict(self, X):
        return self.model.predict(X)

# Create an instance of the KerasRegressorWrapper
keras_reg = KerasRegressorWrapper()


In [203]:
keras_reg.fit(X_train, y_train, epochs=100,
              # fill in code
              validation_data=( X_valid,y_valid ),
              callbacks=[keras.callbacks.EarlyStopping(patience=10)])

Epoch 1/100
1563/1563 [==============================] - 7s 4ms/step - loss: 0.4428 - accuracy: 0.8475 - val_loss: 0.3589 - val_accuracy: 0.8698
Epoch 2/100
1563/1563 [==============================] - 5s 3ms/step - loss: 0.3097 - accuracy: 0.8887 - val_loss: 0.3173 - val_accuracy: 0.8854
Epoch 3/100
1563/1563 [==============================] - 5s 3ms/step - loss: 0.2677 - accuracy: 0.9051 - val_loss: 0.3118 - val_accuracy: 0.8892
Epoch 4/100
1563/1563 [==============================] - 5s 3ms/step - loss: 0.2382 - accuracy: 0.9165 - val_loss: 0.3054 - val_accuracy: 0.8936
Epoch 5/100
1563/1563 [==============================] - 5s 3ms/step - loss: 0.2142 - accuracy: 0.9240 - val_loss: 0.3041 - val_accuracy: 0.8946
Epoch 6/100
1563/1563 [==============================] - 5s 3ms/step - loss: 0.1942 - accuracy: 0.9308 - val_loss: 0.3035 - val_accuracy: 0.8964
Epoch 7/100
1563/1563 [==============================] - 5s 3ms/step - loss: 0.1765 - accuracy: 0.9385 - val_loss: 0.3118 - val_ac

KerasRegressorWrapper()

In [204]:
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV

param_distribs = {
    # fill in code
    "n_hidden": [ 1, 2 ],
    "n_neurons": np.arange(200 , 500)
}

rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs, n_iter=20, cv=3, verbose=2, n_jobs=-1)
rnd_search_cv.fit(X_train, y_train, epochs=100,
                  # fill in code
                  validation_data=( X_valid , y_valid ),
                  callbacks=[keras.callbacks.EarlyStopping(patience=10)])

Fitting 3 folds for each of 20 candidates, totalling 60 fits


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Epoch 1/100
1563/1563 [==============================] - 6s 3ms/step - loss: 0.4149 - accuracy: 0.8559 - val_loss: 0.3475 - val_accuracy: 0.8770
Epoch 2/100
1563/1563 [==============================] - 4s 3ms/step - loss: 0.2914 - accuracy: 0.8957 - val_loss: 0.3028 - val_accuracy: 0.8912
Epoch 3/100
1563/1563 [==============================] - 5s 3ms/step - loss: 0.2442 - accuracy: 0.9120 - val_loss: 0.3079 - val_accuracy: 0.8892
Epoch 4/100
1563/1563 [==============================] - 5s 3ms/step - loss: 0.2103 - accuracy: 0.9251 - val_loss: 0.3050 - val_accuracy: 0.8920
Epoch 5/100
1563/1563 [==============================] - 5s 3ms/step - loss: 0.1805 - accuracy: 0.9365 - val_loss: 0.3052 - val_accuracy: 0.8966
Epoch 6/100
1563/1563 [==============================] - 5s 3ms/step - loss: 0.1548 - accuracy: 0.9454 - val_loss: 0.3161 - val_accuracy: 0.8960
Epoch 7/100
1563/1563 [==============================] - 5s 3ms/step - loss: 0.1324 - accuracy: 0.9541 - val_loss: 0.3303 - val_ac

RandomizedSearchCV(cv=3, estimator=KerasRegressorWrapper(), n_iter=20,
                   n_jobs=-1,
                   param_distributions={'n_hidden': [1, 2],
                                        'n_neurons': array([200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212,
       213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225,
       226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238,
       239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251,
       252, 253...
       408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 419, 420,
       421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431, 432, 433,
       434, 435, 436, 437, 438, 439, 440, 441, 442, 443, 444, 445, 446,
       447, 448, 449, 450, 451, 452, 453, 454, 455, 456, 457, 458, 459,
       460, 461, 462, 463, 464, 465, 466, 467, 468, 469, 470, 471, 472,
       473, 474, 475, 476, 477, 478, 479, 480, 481, 482, 483, 484, 485,
       486, 487, 488, 489, 490, 491, 492, 493, 494, 495, 496, 497, 498,
       499])},
                   verbose=2)

In [205]:
rnd_search_cv.best_params_

{'n_neurons': 484, 'n_hidden': 1}

Show the summary of the best model obtained from the randomized search. Report the test result using the best model, and compare this result with the previous results. (20 points)

In [206]:
# fill in code
model = rnd_search_cv.best_estimator_.model
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 484)               90992     
                                                                 
 dense_3 (Dense)             (None, 10)                4850      
                                                                 
Total params: 95842 (374.38 KB)
Trainable params: 95842 (374.38 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [207]:
model.evaluate(X_test_reduced, y_test)

313/313 [==============================] - 1s 2ms/step - loss: 0.4565 - accuracy: 0.8910


[0.4564920961856842, 0.890999972820282]

The result from the randomized search ...